# Day 1 Notebook: Predicting Toronto's Weather

## Introduction

Toronto’s weather can be notoriously unpredictable, especially when planning outdoor events. In this project, we’ll build a time-series forecasting model to predict the temperature at least a week or two in advance. By the end of our journey, we’ll integrate this model into a Spring Boot backend, create a React frontend, and deploy everything on AWS S3. But for **Day 1**, our primary goal is to:

1. Load and explore our **Toronto weather dataset**  
2. Clean and manipulate the data as needed  
3. Build a **simple baseline** TensorFlow model (a toy model)  
4. Compile and evaluate this model  

This baseline will serve as our starting point, and we’ll improve upon it in the upcoming days (e.g., adding time-series techniques like LSTM).

---

## Lazy Programmer’s Approach

1. Load the data  
2. Manipulate the data  
3. Build the model  
4. Compile the model  
5. Evaluate the model  

We’ll follow these steps in this notebook. Once completed, we’ll have our initial results ready to **push to GitHub**.

---

In [70]:

## 0. Imports & Setup
# Install missing libraries
%pip install pandas numpy matplotlib tensorflow scikit-learn

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# For scikit-learn baseline or metrics if needed
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Make matplotlib plots inline
%matplotlib inline

print("Notebook environment set up. TensorFlow version:", tf.__version__)


Note: you may need to restart the kernel to use updated packages.
Notebook environment set up. TensorFlow version: 2.19.0



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\wafee\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Load In Data From https://toronto.weatherstats.ca
The data provided is a CSV  (10K Rows) of various weather details based in Toronto in the past 20 or so years. 
We will: 
* Clean and Prepare the Data for our Simple Model (Linear Regression)
* Load Our Data Into A Pandas DataFrame (CSV)
* Drop Irrelevant/Empty/Unfinished Columns from the DataFrame For Our Purposes
* Fill in any columns of interest with the mean (if they have minimal unfinished entries)
* Sort the Rows By Date For Our Time Series
* Define Basic Features and Target Values

In [71]:
#Lets Load Our Data, and Get Some Cleaning And Preprocessing Done
# Load the dataset

df = pd.read_csv("./DailyWeatherStats.csv", parse_dates=['date'])
print("Data Loaded, Shape:", df.shape)
df.head()

Data Loaded, Shape: (10000, 72)


C:\Users\wafee\AppData\Local\Temp\ipykernel_30024\3248805573.py:4: DtypeWarning: Columns (46,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./DailyWeatherStats.csv", parse_dates=['date'])


,date,max_temperature,avg_hourly_temperature,avg_temperature,min_temperature,max_humidex,min_windchill,max_relative_humidity,avg_hourly_relative_humidity,avg_relative_humidity,...,avg_cloud_cover_4,min_cloud_cover_4,max_cloud_cover_8,avg_hourly_cloud_cover_8,avg_cloud_cover_8,min_cloud_cover_8,max_cloud_cover_10,avg_hourly_cloud_cover_10,avg_cloud_cover_10,min_cloud_cover_10
0,2025-04-04,11.0,5.79,5.90,0.8,NaN,NaN,73,56.5,57.0,...,NaN,NaN,8.0,4.6,4.5,1.0,NaN,NaN,NaN,NaN
1,2025-04-03,20.0,11.41,11.80,3.6,NaN,NaN,100,73.1,72.0,...,NaN,NaN,8.0,6.7,5.5,3.0,NaN,NaN,NaN,NaN
2,2025-04-02,4.7,0.18,1.50,-1.7,NaN,-9.0,100,77.0,70.0,...,NaN,NaN,8.0,8.0,7.5,7.0,NaN,NaN,NaN,NaN
3,2025-04-01,3.7,0.79,0.90,-1.9,NaN,-6.0,79,48.8,53.5,...,NaN,NaN,8.0,3.9,4.5,1.0,NaN,NaN,NaN,NaN
4,2025-03-31,12.0,5.16,6.25,0.5,NaN,NaN,100,82.4,81.0,...,NaN,NaN,8.0,7.6,5.0,2.0,NaN,NaN,NaN,NaN


## Identify Columns With Missing Values (NaN)

In [72]:
#Lets Check and Clean Missing Values
#Validate Data Struucture, and Remove any Unwanted Columns 
df.info()
df.isna().sum().sort_values(ascending=False).head(20)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 72 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   date                           10000 non-null  datetime64[ns]
 1   max_temperature                10000 non-null  float64       
 2   avg_hourly_temperature         10000 non-null  float64       
 3   avg_temperature                10000 non-null  float64       
 4   min_temperature                10000 non-null  float64       
 5   max_humidex                    2837 non-null   float64       
 6   min_windchill                  3309 non-null   float64       
 7   max_relative_humidity          10000 non-null  int64         
 8   avg_hourly_relative_humidity   10000 non-null  float64       
 9   avg_relative_humidity          10000 non-null  float64       
 10  min_relative_humidity          10000 non-null  int64         
 11  max_dew_point   

solar_radiation              10000
avg_hourly_cloud_cover_4     10000
avg_cloud_cover_4            10000
max_cloud_cover_4            10000
min_cloud_cover_4            10000
avg_hourly_cloud_cover_10     9763
avg_cloud_cover_10            9763
min_cloud_cover_10            9763
max_cloud_cover_10            9763
max_health_index              9760
avg_health_index              9760
avg_hourly_health_index       9760
min_health_index              9760
max_humidex                   7163
min_windchill                 6691
min_uv_forecast               6562
max_uv_forecast               6562
sunset_unixtime               5888
sunrise_unixtime              5888
sunset_hhmm                   5888
dtype: int64

Lets keep columns with info that we're interested in, or that are filled with values (good data)
Lets also fill in any missing data in our columns of interest with the mean data. 

In [73]:
#Drop Columns w/ Non Null Values, and Values We Aren't Interested In for Our Model 
keep_cols = [
    "date",
    "max_temperature",
    "avg_hourly_temperature",
    "avg_temperature",
    "min_temperature",
    "max_relative_humidity",
    "avg_hourly_relative_humidity",
    "avg_relative_humidity",
    "min_relative_humidity",
    "max_dew_point",
    "avg_hourly_dew_point",
    "avg_dew_point",
    "min_dew_point",
    "max_wind_speed",
    "avg_hourly_wind_speed",
    "avg_wind_speed",
    "min_wind_speed",
    "max_pressure_sea",
    "avg_hourly_pressure_sea",
    "avg_pressure_sea",
    "min_pressure_sea",
    "max_pressure_station",
    "avg_hourly_pressure_station",
    "avg_pressure_station",
    "min_pressure_station",
    "max_visibility",
    "avg_hourly_visibility",
    "avg_visibility",
    "min_visibility",
    "heatdegdays",
    "cooldegdays",
    "growdegdays_5",
    "growdegdays_7",
    "growdegdays_10",
    "precipitation",     # 9974 non-null
    "rain",              # 9962 non-null
    "snow"               # 9972 non-null
]
df = df[keep_cols]
df.isna().sum()

#Fill Missing Values with mean of the column
df['precipitation'].fillna(df['precipitation'].mean(), inplace=True) 
df['rain'].fillna(df['rain'].mean(), inplace=True) 
df['snow'].fillna(df['snow'].mean(), inplace=True) 
df.isna().sum()

C:\Users\wafee\AppData\Local\Temp\ipykernel_30024\696592791.py:45: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['precipitation'].fillna(df['precipitation'].mean(), inplace=True)
C:\Users\wafee\AppData\Local\Temp\ipykernel_30024\696592791.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves

date                            0
max_temperature                 0
avg_hourly_temperature          0
avg_temperature                 0
min_temperature                 0
max_relative_humidity           0
avg_hourly_relative_humidity    0
avg_relative_humidity           0
min_relative_humidity           0
max_dew_point                   0
avg_hourly_dew_point            0
avg_dew_point                   0
min_dew_point                   0
max_wind_speed                  0
avg_hourly_wind_speed           0
avg_wind_speed                  0
min_wind_speed                  0
max_pressure_sea                0
avg_hourly_pressure_sea         0
avg_pressure_sea                0
min_pressure_sea                0
max_pressure_station            0
avg_hourly_pressure_station     0
avg_pressure_station            0
min_pressure_station            0
max_visibility                  0
avg_hourly_visibility           0
avg_visibility                  0
min_visibility                  0
heatdegdays   

In [74]:
df.info() #Verify Data Cleanup

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 37 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   date                          10000 non-null  datetime64[ns]
 1   max_temperature               10000 non-null  float64       
 2   avg_hourly_temperature        10000 non-null  float64       
 3   avg_temperature               10000 non-null  float64       
 4   min_temperature               10000 non-null  float64       
 5   max_relative_humidity         10000 non-null  int64         
 6   avg_hourly_relative_humidity  10000 non-null  float64       
 7   avg_relative_humidity         10000 non-null  float64       
 8   min_relative_humidity         10000 non-null  int64         
 9   max_dew_point                 10000 non-null  float64       
 10  avg_hourly_dew_point          10000 non-null  float64       
 11  avg_dew_point                

In [75]:
#Lets Sort Our Data By Date, Since We're Going to be implmeenting a Time Series Model Eventually
df = df.sort_values(by='date').reset_index(drop=True)
df.head(10) #Check Sorted Data

#Feature and Target Defintion, for this baseline model we'll do something simple. 
df['DayOfYear'] = df['date'].dt.dayofyear
X = df[['DayOfYear']] #Minimal Feature Set for Baseline Model
Y = df['avg_temperature'] #Target Variable

## Split Our Data into Training & Test Sets for Simple Model
We'll do an 80/20 split on our dataframe for training and testing using scikit learn's train test split function. This means that we'll train our simple model based on the first 80% of our data and test it on the last 20. Recall that we sorted our data chronologically. 


In [76]:
X_Train, X_test, Y_Train, Y_test = train_test_split(X,Y, test_size=0.2, shuffle=False) #Split Data into Train and Test Sets, No Shuffle for Time Series 80/20 Split

print("X Train Shape:", X_Train.shape, "Y Train Shape:", Y_Train.shape)

X Train Shape: (8000, 1) Y Train Shape: (8000,)


Lets use the Keras API to build a sequential model, our model will contain two hidden layers, where each layer is a dense layer which means every neuron in this layer connects to the inputs  in the previous
* Dense Layer 1: Activation Function Relu - The RELU is a nonlinear model that allows the model to learn complex relationships, instead of a purely linear relationship, avoids the vanishing gradient problem during gradient descent
* Dense Layer 2: We're predicting a single numeric temperature, with no activation function since its regression. 

In [77]:
#Lets use the Keras API to build a simple Sequential Model, and Train it on our Data 
# Build a simple Sequential model
model = keras.Sequential([ 
    layers.Dense(16, activation='relu', input_shape=(X_Train.shape[1],)), #Input Layer
    layers.Dense(1), #Single Output Layer
]) 

C:\Users\wafee\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Compile The Model With Default Optimizer Adam, and Mean Squared Error Loss Function
* Adam - A good default for various tasks
* MSE - Appropriate for regression problems, as it helps the model fit data by reducing large errors
* MAE - Judgement by mean absolute error

In [78]:
model.compile(
    optimizer='adam',
    loss='mse',
    metrics=['mae']
)
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_10 (Dense)                │ (None, 16)             │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49 (196.00 B)

 Trainable params: 49 (196.00 B)

 Non-trainable params: 0 (0.00 B)

In [69]:
history = model.fit(
    X_Train, Y_Train,
    epochs=200,       # Adjustable based on training time and performance, can increased if we're underfitting
    batch_size=32,
    validation_split=0.1, #Take 10% of the training data for validation
    verbose=1
)


Epoch 1/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 1385.0012 - mae: 27.3586 - val_loss: 114.7175 - val_mae: 9.0290
Epoch 2/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 806us/step - loss: 110.2372 - mae: 8.8314 - val_loss: 114.5256 - val_mae: 9.0074
Epoch 3/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 818us/step - loss: 107.0770 - mae: 8.7263 - val_loss: 114.3563 - val_mae: 8.9845
Epoch 4/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: 107.4987 - mae: 8.7580 - val_loss: 114.6258 - val_mae: 9.0413
Epoch 5/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 779us/step - loss: 106.5424 - mae: 8.7148 - val_loss: 114.1331 - val_mae: 8.9604
Epoch 6/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 799us/step - loss: 108.6401 - mae: 8.8158 - val_loss: 114.0536 - val_mae: 8.9821
Epoch 7/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - loss: 105.3696 - mae: 8.6721 - val_loss: 113.9265 - val_mae: 8.9411
Epoch 8/200
225/225 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step - loss: 106.6610 - mae: 8.7135 - val_loss: 113.7923 - val_mae: 8.9433


### After 200 Epochs of Training, lets Evaluate Our Simple Regression Model
### Evaluate Our Model On The Test Set



In [81]:
y_pred = model.predict(X_test).flatten()

mse = mean_squared_error(Y_test, y_pred)
mae = mean_absolute_error(Y_test, y_pred)
rmse = np.sqrt(mse)

print(f"Test MAE: {mae:.2f} °C")
print(f"Test RMSE: {rmse:.2f} °C")


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 509us/step
Test MAE: 85.61 °C
Test RMSE: 98.77 °C
